In [1]:
import numpy as np
import matplotlib.pylab as plt
import pickle

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

本章的主题是卷积神经网络（Convolutional Neural Network，CNN）。

CNN被用于图像识别、语音识别等各种场合，在图像识别的比赛中，基于深度学习的方法几乎都以CNN为基础。

# 整体结构

CNN和之前介绍的神经网络一样，可以像乐高积木一样通过组装层来构建。不过，CNN中新出现了卷积层（Convolution层）和池化层（Pooling层）。

<img src="img/7_1.png" alt="Drawing" style="width: 500px;"/>

- CNN 的层的连接顺序是“Convolution - ReLU -（Pooling）”（Pooling层有时会被省略）。

- 这可以理解为之前的“Affine - ReLU”连接被替换成了“Convolution - ReLU -（Pooling）”连接。

- 还需要注意的是，在图7-2的CNN中，靠近输出的层中使用了之前的“Affine - ReLU”组合。

- 此外，最后的输出层中使用了之前的“Affine - Softmax”组合。

这些都是一般的CNN中比较常见的结构。


# 卷积层

**为什么要卷积层？**

之前介绍的全连接的神经网络中使用了全连接层（Affine层）。在全连接层中，相邻层的神经元全部连接在一起。

但是，全连接层存在什么问题呢？那就是数据的形状被“忽视”了。

比如，输入数据是图像时，图像通常是高、长、通道方向上的3维形状。但是，向全连接层输入时，需要将3维数据拉平为1维数据。

前面提到的使用了MNIST数据集的例子中，输入图像就是1通道、高28像素、长28像素的（1, 28, 28）形状，但却被排成1列，以784个数据的形式输入到最开始的Affine层。

图像是3维形状，这个形状中应该含有重要的空间信息。

3维形状中可能隐藏有值得提取的本质模式。但是，因为全连接层会忽视形状，将全部的输入数据作为相同的神经元（同一维度的神经元）处理，所以无法利用与形状相关的信息。

**而卷积层可以保持形状不变。当输入数据是图像时，卷积层会以3维数据的形式接收输入数据，并同样以3维数据的形式输出至下一层。因此，在CNN中，可以（有可能）正确理解图像等具有形状的数据。**

## 卷积运算

卷积层进行的处理就是卷积运算。卷积运算相当于图像处理中的“滤波器运算”。

<img src="img/7_3.png" alt="Drawing" style="width: 500px;"/>

如图7-3所示，卷积运算对输入数据应用滤波器。在这个例子中，输入数据是有高长方向的形状的数据，滤波器也一样，有高长方向上的维度。假设用（height, width）表示数据和滤波器的形状，则在本例中，输入大小是(4, 4)，滤波器大小是(3, 3)，输出大小是(2, 2)。另外，有的文献中也会用“核”这个词来表示这里所说的“滤波器”。

对于输入数据，卷积运算以一定间隔滑动滤波器的窗口并应用。
将各个位置上滤波器的元素和输入的对应元素相乘，然后再求和（有时将这个计算称为乘积累加运算）。然后，将这个结果保存到输出的对应位置。将这个过程在所有位置都进行一遍，就可以得到卷积运算的输出。

<img src="img/7_4.png" alt="Drawing" style="width: 500px;"/>


CNN中也存在偏置。
如图7-5所示，向应用了滤波器的数据加上了偏置。偏置通常只有1个 （1 × 1）（本例中，相对于应用了滤波器的4个数据，偏置只有1个），这个值会被加到应用了滤波器的所有元素上。
<img src="img/7_5.png" alt="Drawing" style="width: 500px;"/>



## 填充

- 在进行卷积层的处理之前，有时要向输入数据的周围填入固定的数据（比如0等），这称为填充（padding），是卷积运算中经常会用到的处理。

- 使用填充主要是为了调整输出的大小。比如图7-5，对大小为(4, 4)的输入数据应用(3, 3)的滤波器时，输出大小变为(2, 2)，相当于输出大小比输入大小缩小了 2个元素。这在反复进行多次卷积运算的深度网络中会成为问题。为什么呢？因为如果每次进行卷积运算都会缩小空间，那么在某个时刻输出大小就有可能变为 1，导致无法再应用卷积运算。为了避免出现这样的情况，就要使用填充。

<img src="img/7_6.png" alt="Drawing" style="width: 500px;"/>
在图7-6的例子中，对大小为(4, 4)的输入数据应用了幅度为1的填充。“幅度为1的填充”是指用幅度为1像素的0填充周围。

通过填充，大小为(4, 4)的输入数据变成了(6, 6)的形状。然后，应用大小为(3, 3)的滤波器，生成了大小为(4, 4)的输出数据。这个例子中将填充设成了1，不过填充的值也可以设置成2、3等任意的整数。在图7-5的例子中，如果将填充设为2，则输入数据的大小变为(8, 8)；如果将填充设为3，则大小变为(10, 10)。

## 步幅

应用滤波器的位置间隔称为步幅（stride）。之前的例子中步幅都是1，如果将步幅设为2，则如图7-7所示，应用滤波器的窗口的间隔变为2个元素。
<img src="img/7_7.png" alt="Drawing" style="width: 500px;"/>




**综上，**
- 增大步幅后，输出大小会变小。
- 而增大填充后，输出大小会变大。

## 三维数据的卷积运算

对于3维数据，除了高、长方向之外，还需要处理通道方向。以下为具体的卷积运算的例子。

<img src="img/7_8.png" alt="Drawing" style="width: 500px;"/>

和2维数据时（图7-3的例子）相比，可以发现纵深方向（通道方向）上特征图增加了。通道方向上有多个特征图时，会按通道进行输入数据和滤波器的卷积运算，并将结果相加，从而得到输出。

需要注意的是，在3维数据的卷积运算中，输入数据和滤波器的通道数要设为相同的值。

在这个例子中，输入数据和滤波器的通道数一致，均为3。
滤波器大小可以设定为任意值（不过，每个通道的滤波器大小要全部相同）。这个例子中滤波器大小为(3, 3)，但也可以设定为(2, 2)、(1, 1)、(5, 5)等任意值。

<img src="img/7_9.png" alt="Drawing" style="width: 500px;"/>




# 池化层

**为什么要有池化层？**

- 这一层有时候确实会被省略。

**先来看看实现**，为什么存在现在我还回答不来~

- 池化是缩小高、长方向上的空间的运算。比如，如图7-14所示，进行将2 × 2的区域集约成1个元素的处理，缩小空间大小。
- 下图的运算展示的是 “Max池化”，如图所示，从2 × 2的区域中取出最大的元素。

<img src="img/7_14.png" alt="Drawing" style="width: 500px;"/>

- 此外，这个例子中将步幅设为了2，所以2 × 2的窗口的移动间隔为2个元素。
- 另外，一般来说，池化的窗口大小会和步幅设定成相同的值。比如，3 × 3的窗口的步幅会设为3，4 × 4的窗口的步幅会设为4等。
- 除了Max池化之外，还有Average池化等。相对于Max池化是从目标区域中取出最大值，Average池化则是计算目标区域的平均值。在图像识别领域，主要使用Max池化。后文说的“池化层”，均指的是Max池化。


**池化层的特征**

1. 没有要学习的参数

池化层和卷积层不同，没有要学习的参数。池化只是从目标区域中取最大值（或者平均值），所以不存在要学习的参数。

2. 通道数不发生变化

经过池化运算，输入数据和输出数据的通道数不会发生变化。如图7-15所示，计算是按通道独立进行的。
<img src="img/7_15.png" alt="Drawing" style="width: 500px;"/>


3. 对微小的位置变化具有鲁棒性（健壮）

输入数据发生微小偏差时，池化仍会返回相同的结果。因此，池化对输入数据的微小偏差具有鲁棒性。比如，3 × 3的池化的情况下，如图7-16所示，池化会吸收输入数据的偏差（根据数据的不同，结果有可能不一致）。

<img src="img/7_16.png" alt="Drawing" style="width: 500px;"/>


In [2]:
# 7.4 卷积层和池化层的实现

In [3]:
# CNN中各层间传递的数据是4维数据。所谓4维数据，比如
# 数据的形状是(10, 1, 28, 28)，则它对应10个高为28、长为28、通道为1的数
# 据。用Python来实现的话，如下所示。

In [4]:
import numpy as np
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [ ]:
# 如果要访问第1个数据，只要写x[0]就可以了（注意Python的索
# 引是从0开始的）。同样地，用x[1]可以访问第2个数据。

In [11]:
x[0].shape

(1, 28, 28)

In [12]:
x[0]

array([[[0.33422283, 0.25017584, 0.10998635, 0.98133287, 0.4564739 ,
         0.60860829, 0.72935937, 0.66182336, 0.00830906, 0.32200186,
         0.68526191, 0.41233357, 0.98638851, 0.56388968, 0.77847805,
         0.61996636, 0.77045825, 0.61165972, 0.99565704, 0.5936943 ,
         0.62735811, 0.29561036, 0.6126446 , 0.33312011, 0.71034213,
         0.3707909 , 0.78017753, 0.01974643],
        [0.65750555, 0.16993749, 0.08097377, 0.83609296, 0.52948281,
         0.66646797, 0.84111156, 0.1501675 , 0.05444247, 0.68799729,
         0.01151102, 0.11954533, 0.49806426, 0.80024149, 0.59143208,
         0.33735674, 0.62703941, 0.49377166, 0.1450928 , 0.08141418,
         0.31204156, 0.7176734 , 0.61790289, 0.38790059, 0.42274397,
         0.95762967, 0.1306233 , 0.07289688],
        [0.07566849, 0.96522596, 0.62051577, 0.19302344, 0.39374249,
         0.07661005, 0.77216253, 0.73041675, 0.38789843, 0.66691874,
         0.41794234, 0.38896464, 0.10118004, 0.26104856, 0.67891512,
         0.

In [13]:
# 如果要访问第1个数据的第1个通道的空间数据，可以写成下面这样。
x[0, 0] # 或者x[0][0]

array([[0.33422283, 0.25017584, 0.10998635, 0.98133287, 0.4564739 ,
        0.60860829, 0.72935937, 0.66182336, 0.00830906, 0.32200186,
        0.68526191, 0.41233357, 0.98638851, 0.56388968, 0.77847805,
        0.61996636, 0.77045825, 0.61165972, 0.99565704, 0.5936943 ,
        0.62735811, 0.29561036, 0.6126446 , 0.33312011, 0.71034213,
        0.3707909 , 0.78017753, 0.01974643],
       [0.65750555, 0.16993749, 0.08097377, 0.83609296, 0.52948281,
        0.66646797, 0.84111156, 0.1501675 , 0.05444247, 0.68799729,
        0.01151102, 0.11954533, 0.49806426, 0.80024149, 0.59143208,
        0.33735674, 0.62703941, 0.49377166, 0.1450928 , 0.08141418,
        0.31204156, 0.7176734 , 0.61790289, 0.38790059, 0.42274397,
        0.95762967, 0.1306233 , 0.07289688],
       [0.07566849, 0.96522596, 0.62051577, 0.19302344, 0.39374249,
        0.07661005, 0.77216253, 0.73041675, 0.38789843, 0.66691874,
        0.41794234, 0.38896464, 0.10118004, 0.26104856, 0.67891512,
        0.6547055 , 0.6601

In [14]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

In [15]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75)

(9, 75)


In [20]:
x1[0, 0]

array([[0.03839591, 0.24788389, 0.30235029, 0.08191092, 0.41408943,
        0.01270931, 0.12752221],
       [0.71850661, 0.75066788, 0.45521606, 0.9317958 , 0.17290206,
        0.66372715, 0.17402088],
       [0.2954876 , 0.48134555, 0.13000129, 0.05147708, 0.64526114,
        0.38252742, 0.80410039],
       [0.38144241, 0.80397651, 0.24307599, 0.62918336, 0.94118704,
        0.38737915, 0.09172245],
       [0.98844763, 0.02761019, 0.5893923 , 0.75958281, 0.85564032,
        0.00177706, 0.87044051],
       [0.55277764, 0.29453557, 0.70165366, 0.03315095, 0.61735683,
        0.91846378, 0.18907312],
       [0.23316863, 0.38232395, 0.24437651, 0.59393741, 0.52680464,
        0.55943772, 0.67311005]])

In [17]:
col1[0]

array([0.03839591, 0.24788389, 0.30235029, 0.08191092, 0.41408943,
       0.71850661, 0.75066788, 0.45521606, 0.9317958 , 0.17290206,
       0.2954876 , 0.48134555, 0.13000129, 0.05147708, 0.64526114,
       0.38144241, 0.80397651, 0.24307599, 0.62918336, 0.94118704,
       0.98844763, 0.02761019, 0.5893923 , 0.75958281, 0.85564032,
       0.89385831, 0.25544756, 0.27892884, 0.31122035, 0.21801504,
       0.11792877, 0.48624226, 0.47540864, 0.70697923, 0.53771164,
       0.63178935, 0.03395237, 0.54431881, 0.26075442, 0.97089694,
       0.22184166, 0.04115981, 0.8805996 , 0.55990329, 0.0829363 ,
       0.58685257, 0.13151869, 0.53379512, 0.24328321, 0.79169923,
       0.77360492, 0.68987932, 0.41297011, 0.65934844, 0.13740372,
       0.87412263, 0.21571147, 0.18482165, 0.45983048, 0.56971224,
       0.0475127 , 0.74434783, 0.79941088, 0.22791843, 0.83375254,
       0.52459545, 0.60625093, 0.65949355, 0.10853401, 0.93656728,
       0.22922065, 0.63441701, 0.90290005, 0.4122449 , 0.43182

In [21]:
x2 = np.random.rand(10, 3, 7, 7) # 10个数据
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # (90, 75)

(90, 75)


In [ ]:
# 使用im2col来实现卷积层。

In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 滤波器的展开
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        return out

In [22]:
# 池化层的实现

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 展开(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 最大值(2)
        out = np.max(col, axis=1)
        
        # 转换(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        return out

In [ ]:
# 7.5 CNN的实现

首先来看一下SimpleConvNet的初始化（__init__），取下面这些参数。
参数
- input_dim―输入数据的维度：（通道，高，长） 
- conv_param―卷积层的超参数（字典）。字典的关键字如下：
    - filter_num―滤波器的数量
    - filter_size―滤波器的大小
    - stride―步幅
    - pad―填充
- hidden_size―隐藏层（全连接）的神经元数量
- output_size―输出层（全连接）的神经元数量
- weitght_int_std―初始化时权重的标准差

In [23]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param={'filter_num':30, 'filter_size':5,'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / \
        filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) *(conv_output_size/2))
        
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_siz
                                     
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],self.params['b1'],conv_param['stride'],conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'],self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'],self.params['b3'])
        self.last_layer = softmaxwithloss()
                             
  